## Train a machine learning model using prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.
1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

Import TensorFlow

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.13.0


## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


## Build a machine learning model
Build a `tf.keras.Sequential` model:

In [3]:
model =  tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.1278486 , -0.07632111, -0.97122043, -0.6982423 , -0.48008153,
        -0.26463985, -1.1206989 ,  0.6135393 ,  0.81031936, -0.9337252 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to probabilities for each class

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.12433167, 0.10137059, 0.04142496, 0.05442708, 0.0676957 ,
        0.08397039, 0.03567339, 0.20207591, 0.24602263, 0.04300769]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [9]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.4`.

In [10]:
loss_fn(y_train[:1],predictions).numpy()

2.477291

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [11]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and mimize the loss:

In [12]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2964 - accuracy: 0.9140
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1429 - accuracy: 0.9582
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1076 - accuracy: 0.9670
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0873 - accuracy: 0.9726
Epoch 5/5
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0745 - accuracy: 0.9768


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [13]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0750 - accuracy: 0.9771 - 1s/epoch - 4ms/step


[0.07498466968536377, 0.9771000146865845]

The image classifier is now trained to ~98% accuracy on this dataset.

In [14]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.00843645e-08, 1.27340460e-09, 1.32750344e-07, 3.04487039e-05,
        1.07716364e-10, 1.45804407e-07, 3.26322763e-13, 9.99961257e-01,
        8.26944586e-07, 7.31546334e-06],
       [6.96532834e-06, 2.08938238e-03, 9.97822523e-01, 7.34243877e-05,
        2.88310872e-14, 3.17894865e-06, 6.12397628e-07, 8.65502947e-11,
        3.88784110e-06, 6.44007070e-11],
       [6.17909279e-08, 9.98262823e-01, 1.25937600e-04, 7.08831840e-06,
        2.02668998e-05, 1.01388005e-05, 6.42308351e-06, 1.41926133e-03,
        1.47818573e-04, 2.58111697e-07],
       [9.99883652e-01, 7.23274940e-10, 2.19541698e-06, 3.84488743e-08,
        3.00837087e-07, 4.32687875e-06, 1.02278907e-04, 4.43735189e-06,
        2.21079310e-09, 2.75051502e-06],
       [1.23598857e-05, 4.07041484e-10, 1.53046039e-05, 3.63201522e-08,
        9.95207489e-01, 2.45498285e-07, 3.71556380e-05, 2.32893544e-05,
        9.90848730e-06, 4.69417498e-03]], dtype=float32)>

# Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the **[Keras](https://www.tensorflow.org/guide/keras/overview) API.**
